LEVEL-2: ---
We are now going to Level-2 of our trading system 

In this notebook, I will be: 
1. Optimizing the trading strategy logic by incorporating additional indicators such as RSI (Relative Strength Index) 
   or MACD (Moving Average Convergence/Divergence) 

2. Add logic to avoid market conditions such as volatility filters, in order to avoid false signals. 

3. I'll also implement try-except blocks to manage API errors (e.g., Alpaca outages) and add logging to track trades and performance.

In [ ]:
import asyncio
import platform
import alpaca_trade_api as tradeapi
import pandas as pd
import logging
import numpy as np
FPS = 60

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Replace with your Alpaca API keys
API_KEY = 'PKMSXCZ5JFNRILSSDE7Z'
SECRET_KEY = '0xfaUPAQEifIEaSZbqOuUUEGlP4GJsa0GEAj5vHC'
BASE_URL = 'https://paper-api.alpaca.markets'

async def main():
    api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')
    symbol = 'AAPL'

    def calculate_indicators(data):
        try:
            if len(data) < 50:  
                logger.warning(f"Insufficient data: {len(data)} bars")
                return None
            
            # Moving Averages
            data['short_mavg'] = data['close'].rolling(window=20, min_periods=1).mean()
            data['long_mavg'] = data['close'].rolling(window=50, min_periods=1).mean()
            
            # RSI
            delta = data['close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14, min_periods=1).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14, min_periods=1).mean()
            rs = gain / loss.replace(0, 1e-10)  
            data['rsi'] = 100 - (100 / (1 + rs))
            
            # MACD
            exp1 = data['close'].ewm(span=12, adjust=False).mean()
            exp2 = data['close'].ewm(span=26, adjust=False).mean()
            macd = exp1 - exp2
            data['macd'] = macd
            data['signal'] = macd.ewm(span=9, adjust=False).mean()
            
            return data
        except Exception as e:
            logger.error(f"Error calculating indicators: {str(e)}")
            return None

    def update_loop():
        try:
            # Fetch data
            barset = api.get_bars(symbol, '1D', limit=200).df  
            if barset.empty:
                logger.warning("No data received")
                return

            # Calculate indicators
            data = calculate_indicators(barset)
            if data is None or len(data) < 50:
                return

            latest = data.iloc[-1]
            volatility = data['close'].pct_change().rolling(window=20, min_periods=1).std() * np.sqrt(252)
            current_vol = volatility.iloc[-1] if not np.isnan(volatility.iloc[-1]) else 0.0

            # Trading logic with filters
            if (latest['short_mavg'] > latest['long_mavg'] and 
                latest['rsi'] < 70 and 
                latest['macd'] > latest['signal'] and 
                current_vol < 0.3):
                logger.info("Buy signal triggered")
                api.submit_order(symbol, 10, 'buy', 'market', 'day')
            elif (latest['short_mavg'] < latest['long_mavg'] and 
                  latest['rsi'] > 30 and 
                  latest['macd'] < latest['signal'] and 
                  current_vol < 0.3):
                logger.info("Sell signal triggered")
                api.submit_order(symbol, 10, 'sell', 'market', 'day')
            else:
                logger.info("No action: Conditions not met")
        except Exception as e:
            logger.error(f"Error in update_loop: {e}")

    def setup():
        logger.info(f"Starting trading strategy for {symbol}")

    setup()
    while True:
        update_loop()
        asyncio.sleep(1.0 / FPS)  

if platform.system() == "Emscripten":
    asyncio.ensure_future(main())
else:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        asyncio.ensure_future(main())
    else:
        loop.run_until_complete(main())

2025-07-19 00:39:28,931 - INFO - Starting trading strategy for AAPL
2025-07-19 00:39:29,581 - WARNING - Insufficient data: 1 bars
C:\Users\SOMEN\AppData\Local\Temp\ipykernel_19408\3064167156.py:92: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(1.0 / FPS)  # Non-async sleep for simplicity
2025-07-19 00:39:29,820 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:30,029 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:30,237 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:30,444 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:30,644 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:30,845 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:31,049 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:31,259 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:31,463 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:31,670 - WARNING - Insufficient data: 1 bars
2025-07-19 00:39:31,871 - WARNING - Insufficient data: 1 bars
2025-07